In [1]:
%cd ..

/home/haryoaw/documents/courses/nlp802/project/texteditalay


In [2]:
import pandas as pd

In [3]:
chat_gpt_data = ["data/pred/lexicol_chat-gpt.txt", "data/pred/stif-chat-gpt.txt"]
test_data = ["data/scolid/test.csv", "data/stif_indo/test_with_pointing.csv"]


In [4]:
df_series = []
for file in chat_gpt_data:
    with open(file, 'r+') as file:
        chat_gpt_read = file.read().split('====')
    chat_gpt_read = [line.strip() for line in chat_gpt_read]
    chat_gpt_read = [x.split('\n')[1] for x in chat_gpt_read if x != '']
    chat_gpt_read = [ x.replace('<<', '').replace('>>', '') for x in chat_gpt_read]
    df_series.append(pd.Series(chat_gpt_read))

In [5]:
series_lexicol_chatgpt, series_stif_chatgpt = df_series

In [6]:
moses_data = ["data/pred/lexicol_moses.txt", "data/pred/stif-indo_moses.txt"]
df_series_moses = []
for file in moses_data:
    with open(file, 'r+') as file:
        moses_read = file.read().split('\n')
    df_series_moses.append(pd.Series(moses_read))

In [7]:
series_lexicol_moses, series_stif_moses = df_series_moses

In [8]:
stif_indo_data = [
    "data/pred/stif-indo_felix-scratch.csv",
    "data/pred/stif-indo_felix.csv",
    "data/pred/stif-indo_indobart.csv",
    "data/pred/stif-indo_scratch.csv"
]

In [9]:
pred_series = {}
for file in stif_indo_data:
    # extract file name
    file_name = file.split('/')[-1].split('.')[0]
    stif_data = pd.read_csv(file)
    pred_series[file_name] = pd.Series(stif_data['formal_pred'])

In [10]:
df_stif_eval = pd.DataFrame(pred_series)

In [11]:
df_stif_eval['stif-indo_moses'] = series_stif_moses
df_stif_eval['stif-indo_chat-gpt'] = series_stif_chatgpt

In [12]:
df_test = pd.read_csv("data/stif_indo/test_with_pointing.csv")

In [13]:
import sacrebleu

In [14]:
df_stif_eval['copy'] = df_test['informal']

In [24]:
def manual_detokenize(text):
    # Remove spaces before punctuation
    text = text.replace(" ,", ",").replace(" .", ".").replace(" !", "!").replace(" ?", "?")
    text = text.replace(" ;", ";").replace(" :", ":")

    # Handle contractions in English (if applicable)
    contractions = {"n 't": "n't", " 're": "'re", " 's": "'s", " 'm": "'m", " 'll": "'ll", " 'd": "'d", " 've": "'ve"}
    for key, value in contractions.items():
        text = text.replace(key, value)

    # Additional rules can be added here for other language-specific or tokenization-specific cases

    return text

In [30]:
# iterate over each column
reference = [df_test['formal'].str.lower().apply(manual_detokenize).tolist()]
bleu = sacrebleu.BLEU()

for col in df_stif_eval.columns:
    # check BLEU of it
    candidate = df_stif_eval[col].str.lower().apply(str).apply(manual_detokenize).tolist()
    print(col, bleu.corpus_score(candidate, reference))

stif-indo_felix-scratch BLEU = 1.04 45.0/5.9/0.3/0.0 (BP = 1.000 ratio = 1.413 hyp_len = 8333 ref_len = 5899)
stif-indo_felix BLEU = 39.43 77.8/51.4/35.8/25.6 (BP = 0.901 ratio = 0.906 hyp_len = 5342 ref_len = 5899)
stif-indo_indobart BLEU = 44.00 68.2/49.7/37.8/29.2 (BP = 1.000 ratio = 1.115 hyp_len = 6578 ref_len = 5899)
stif-indo_scratch BLEU = 4.50 12.6/6.5/3.3/1.5 (BP = 1.000 ratio = 5.776 hyp_len = 34074 ref_len = 5899)
stif-indo_moses BLEU = 44.90 75.4/53.0/38.9/28.7 (BP = 0.977 ratio = 0.977 hyp_len = 5763 ref_len = 5899)
stif-indo_chat-gpt BLEU = 37.18 67.0/43.2/30.0/22.0 (BP = 1.000 ratio = 1.029 hyp_len = 6072 ref_len = 5899)
copy BLEU = 32.51 65.3/42.0/28.8/20.4 (BP = 0.913 ratio = 0.916 hyp_len = 5406 ref_len = 5899)


In [22]:
from evaluate import load

bertscore = load("bertscore")


/home/haryoaw/mambaforge/envs/sensei/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [112]:

import numpy as np

for col in df_stif_eval.columns:
    # check BLEU of it
    candidate = df_stif_eval[col].str.lower().apply(str).tolist()
    references = reference[0]
    results = bertscore.compute(
        predictions=candidate, references=references, lang="id"
    )
    print(col, np.mean(results['f1']))

stif-indo_felix-scratch 0.7306181847556563
stif-indo_felix 0.8610941101040065
stif-indo_indobart 0.8700337863165485
stif-indo_scratch 0.6861076763838776
stif-indo_moses 0.8804267728624265
stif-indo_chat-gpt 0.8649380532178012
copy 0.8283023364616163


In [16]:
# DO it for lexicol

pred_series = {}
lexicol_data = [
    "data/pred/lexicol_felix.csv",
    "data/pred/lexicol_indobart_2.csv",
    "data/pred/lexicol_scratch.csv"
]
for file in lexicol_data:
    # extract file name
    file_name = file.split('/')[-1].split('.')[0]
    stif_data = pd.read_csv(file)
    print(stif_data.columns)
    pred_series[file_name] = pd.Series(stif_data['formal_pred' if 'formal_pred' in stif_data.columns else 'formal_predict'])

df_lexicol_eval = pd.DataFrame(pred_series)

df_test_lexicol = pd.read_csv("data/scolid/test.csv")
df_lexicol_eval['lexicol_moses'] = series_lexicol_moses
df_lexicol_eval['lexicol_chat-gpt'] = series_lexicol_chatgpt

df_lexicol_eval['copy'] = df_test_lexicol['informal']


Index(['formal', 'informal', 'formal_pred'], dtype='object')
Index(['informal', 'formal', 'formal_pred'], dtype='object')
Index(['informal', 'formal', 'point_indexes', 'label', 'formal_predict'], dtype='object')


In [31]:
# iterate over each column
reference = [df_test_lexicol['formal'].str.lower().apply(manual_detokenize).tolist()]
bleu = sacrebleu.BLEU()

for col in df_lexicol_eval.columns:
    # check BLEU of it
    candidate = df_lexicol_eval[col].str.lower().apply(str).apply(manual_detokenize).tolist()
    print(col, bleu.corpus_score(candidate, reference))

lexicol_felix BLEU = 17.64 64.4/18.7/10.1/8.0 (BP = 1.000 ratio = 1.099 hyp_len = 4575 ref_len = 4161)
lexicol_indobart_2 BLEU = 48.88 58.3/51.8/46.0/41.1 (BP = 1.000 ratio = 1.427 hyp_len = 5939 ref_len = 4161)
lexicol_scratch BLEU = 0.40 5.9/1.0/0.2/0.0 (BP = 1.000 ratio = 5.494 hyp_len = 22862 ref_len = 4161)
lexicol_moses BLEU = 65.41 75.6/68.1/62.2/57.1 (BP = 1.000 ratio = 1.012 hyp_len = 4210 ref_len = 4161)
lexicol_chat-gpt BLEU = 48.99 71.1/54.4/42.8/34.8 (BP = 1.000 ratio = 1.026 hyp_len = 4268 ref_len = 4161)
copy BLEU = 68.53 84.4/72.6/63.7/56.6 (BP = 0.999 ratio = 0.999 hyp_len = 4158 ref_len = 4161)


In [23]:

import numpy as np

for col in df_lexicol_eval.columns:
    # check BLEU of it
    candidate = df_lexicol_eval[col].str.lower().apply(str).tolist()
    references = reference[0]
    results = bertscore.compute(
        predictions=candidate, references=references, lang="id"
    )
    print(col, np.mean(results['f1']))

lexicol_felix 0.7802248302771121
lexicol_indobart_2 0.8715773867101085
lexicol_scratch 0.6056929048226805
lexicol_moses 0.8961744743950513
lexicol_chat-gpt 0.8771446303445466
copy 0.9275806278598552


In [32]:
# CHeck edit distance

import Levenshtein

In [44]:
from nltk import tokenize

In [48]:
tokenize.wordpunct_tokenize(df_stif_eval['copy'][0])

['belum', 'ada', 'konfirmasi', 'lagi', 'kah', 'min', '?']

In [127]:
# iterate over each column
reference = df_test['formal'].str.lower().apply(manual_detokenize).tolist()
bleu = sacrebleu.BLEU()
levenstein_ops_stif = {}

for col in df_stif_eval.columns:
    if col == 'copy':
        continue
    # check BLEU of it
    candidate = df_stif_eval[col].str.lower().apply(str).apply(manual_detokenize).tolist()
    levenstein_op_col = []
    for i in range(len(candidate)):
        edit_op_found = Levenshtein.opcodes(tokenize.wordpunct_tokenize(df_stif_eval['copy'][i]), tokenize.wordpunct_tokenize(candidate[i]))
        edit_op_found = [x[0] for x in edit_op_found]
        levenstein_op_col.append(edit_op_found)
    levenstein_ops_stif[col] = levenstein_op_col

In [128]:
tokenize.wordpunct_tokenize(df_stif_eval['copy'][55])

['ada', 'apa', 'denganmu', '.', 'xnumberx', 'g', 'loh', 'itu']

In [129]:
tokenize.wordpunct_tokenize(reference[55])

['ada', 'apa', 'denganmu', '?', 'xnumberx', 'g', 'itu', '.']

In [131]:
levenstein_op_col = []

for i in range(len(candidate)):
    edit_op_found = Levenshtein.opcodes(tokenize.wordpunct_tokenize(df_stif_eval['copy'][i]), tokenize.wordpunct_tokenize(reference[i]))
    edit_op_found = [x[0] for x in edit_op_found]
    levenstein_op_col.append(edit_op_found)
levenstein_ops_stif['gold'] = levenstein_op_col

In [132]:
levenstein_ops_stif.keys()

dict_keys(['stif-indo_felix-scratch', 'stif-indo_felix', 'stif-indo_indobart', 'stif-indo_scratch', 'stif-indo_moses', 'stif-indo_chat-gpt', 'gold'])

In [133]:
for key in levenstein_ops_stif.keys():
    print(key)
    op_list = levenstein_ops_stif[key]
    op_counts = []
    # each instance calculate number of insert, delete, replace
    # iterate over each column
    for i in op_list:
        # count number of insert, delete, replace
        insert = i.count('insert')
        delete = i.count('delete')
        replace = i.count('replace')
        keep = i.count('equal')
        op_counts.append((insert, delete, replace, keep))
    df_op_count = pd.DataFrame(op_counts, columns=['insert', 'delete', 'replace', 'keep'])
    print(df_op_count.describe())
    print(df_op_count.sum())

stif-indo_felix-scratch
           insert      delete     replace        keep
count  363.000000  363.000000  363.000000  363.000000
mean     1.867769    0.234160    2.286501    1.831956
std      1.114511    0.449359    0.919710    1.208567
min      0.000000    0.000000    0.000000    0.000000
25%      1.000000    0.000000    2.000000    1.000000
50%      2.000000    0.000000    2.000000    2.000000
75%      2.000000    0.000000    3.000000    2.000000
max      7.000000    2.000000    6.000000    6.000000
insert     678
delete      85
replace    830
keep       665
dtype: int64
stif-indo_felix
           insert      delete     replace        keep
count  363.000000  363.000000  363.000000  363.000000
mean     1.033058    1.212121    2.393939    3.195592
std      0.872545    1.046788    1.375262    1.579490
min      0.000000    0.000000    0.000000    0.000000
25%      0.000000    0.000000    1.000000    2.000000
50%      1.000000    1.000000    2.000000    3.000000
75%      1.000000    2.

In [134]:
# iterate over each column
reference = df_test_lexicol['formal'].str.lower().apply(manual_detokenize).tolist()
bleu = sacrebleu.BLEU()
levenstein_ops_lexicol = {}

for col in df_lexicol_eval.columns:
    if col == 'copy':
        continue
    # check BLEU of it
    candidate = df_lexicol_eval[col].str.lower().apply(str).apply(manual_detokenize).tolist()
    levenstein_op_col = []
    for i in range(len(candidate)):
        edit_op_found = Levenshtein.opcodes(tokenize.wordpunct_tokenize(df_lexicol_eval['copy'][i]), tokenize.wordpunct_tokenize(candidate[i]))
        edit_op_found = [x[0] for x in edit_op_found]
        levenstein_op_col.append(edit_op_found)
    levenstein_ops_lexicol[col] = levenstein_op_col

In [136]:
df_op_count.mean()

insert     0.791837
delete     0.338776
replace    3.461224
keep       3.522449
dtype: float64

In [137]:
levenstein_op_col = []

for i in range(len(candidate)):
    edit_op_found = Levenshtein.opcodes(tokenize.wordpunct_tokenize(df_lexicol_eval['copy'][i]), tokenize.wordpunct_tokenize(reference[i]))
    edit_op_found = [x[0] for x in edit_op_found]
    levenstein_op_col.append(edit_op_found)
levenstein_ops_lexicol['gold'] = levenstein_op_col

In [138]:
for key in levenstein_ops_lexicol.keys():
    print(key)
    op_list = levenstein_ops_lexicol[key]
    op_counts = []
    # each instance calculate number of insert, delete, replace
    # iterate over each column
    for i in op_list:
        # count number of insert, delete, replace
        insert = i.count('insert')
        delete = i.count('delete')
        replace = i.count('replace')
        keep = i.count('equal')
        op_counts.append((insert, delete, replace, keep))
    df_op_count = pd.DataFrame(op_counts, columns=['insert', 'delete', 'replace', 'keep'])
    print(df_op_count.describe())
    print(df_op_count.sum())

lexicol_felix
           insert      delete    replace        keep
count  245.000000  245.000000  245.00000  245.000000
mean     1.069388    0.363265    2.02449    1.281633
std      0.853682    0.764664    3.59066    3.747987
min      0.000000    0.000000    0.00000    0.000000
25%      1.000000    0.000000    1.00000    0.000000
50%      1.000000    0.000000    2.00000    1.000000
75%      1.000000    1.000000    2.00000    2.000000
max      4.000000    9.000000   56.00000   57.000000
insert     262
delete      89
replace    496
keep       314
dtype: int64
lexicol_indobart_2
           insert      delete     replace        keep
count  245.000000  245.000000  245.000000  245.000000
mean     0.881633    0.085714    3.277551    3.004082
std      0.676198    0.532547    2.730465    2.786822
min      0.000000    0.000000    0.000000    0.000000
25%      0.000000    0.000000    2.000000    1.000000
50%      1.000000    0.000000    2.000000    2.000000
75%      1.000000    0.000000    4.0000

BLEU on Operation

In [140]:
levenstein_ops_stif.keys()

dict_keys(['stif-indo_felix-scratch', 'stif-indo_felix', 'stif-indo_indobart', 'stif-indo_scratch', 'stif-indo_moses', 'stif-indo_chat-gpt', 'gold'])

In [143]:
gold_ops_seq = list(map(lambda x: ' '.join(x), levenstein_ops_stif['gold']))
moses_ops_seq = list(map(lambda x: ' '.join(x), levenstein_ops_stif['stif-indo_moses']))
print(bleu.corpus_score(moses_ops_seq, [gold_ops_seq]))

In [145]:
for key in levenstein_ops_stif.keys():
    if key == 'gold':
        continue
    op_seq = list(map(lambda x: ' '.join(x), levenstein_ops_stif[key]))
    print(key, bleu.corpus_score(op_seq, [gold_ops_seq]))

stif-indo_felix-scratch BLEU = 34.40 79.0/63.1/45.5/28.1 (BP = 0.684 ratio = 0.725 hyp_len = 2258 ref_len = 3114)
stif-indo_felix BLEU = 49.75 81.3/64.2/49.3/34.8 (BP = 0.909 ratio = 0.913 hyp_len = 2844 ref_len = 3114)
stif-indo_indobart BLEU = 58.29 84.4/72.9/61.1/48.5 (BP = 0.892 ratio = 0.897 hyp_len = 2794 ref_len = 3114)
stif-indo_scratch BLEU = 40.08 61.9/47.6/35.7/24.6 (BP = 1.000 ratio = 1.448 hyp_len = 4508 ref_len = 3114)
stif-indo_moses BLEU = 57.96 85.5/74.0/60.8/48.4 (BP = 0.882 ratio = 0.889 hyp_len = 2768 ref_len = 3114)
stif-indo_chat-gpt BLEU = 60.60 79.4/67.4/56.0/45.0 (BP = 1.000 ratio = 1.073 hyp_len = 3341 ref_len = 3114)


In [146]:
# Do it for lexicol too
gold_ops_seq_lexicol = list(map(lambda x: ' '.join(x), levenstein_ops_lexicol['gold']))

for key in levenstein_ops_lexicol.keys():
    if key == 'gold':
        continue
    op_seq = list(map(lambda x: ' '.join(x), levenstein_ops_lexicol[key]))
    print(key, bleu.corpus_score(op_seq, [gold_ops_seq_lexicol]))

lexicol_felix BLEU = 31.76 69.4/53.2/43.3/37.5 (BP = 0.642 ratio = 0.693 hyp_len = 1161 ref_len = 1676)
lexicol_indobart_2 BLEU = 70.44 80.1/71.9/66.0/64.8 (BP = 1.000 ratio = 1.060 hyp_len = 1776 ref_len = 1676)
lexicol_scratch BLEU = 24.79 58.4/34.9/15.8/11.8 (BP = 1.000 ratio = 1.159 hyp_len = 1942 ref_len = 1676)
lexicol_moses BLEU = 72.31 86.9/82.8/80.5/77.8 (BP = 0.883 ratio = 0.889 hyp_len = 1490 ref_len = 1676)
lexicol_chat-gpt BLEU = 62.02 75.9/66.9/57.7/50.5 (BP = 1.000 ratio = 1.186 hyp_len = 1988 ref_len = 1676)


In [147]:
# save all of the results
df_stif_eval.to_csv('data/analysis/stif-indo_eval.csv')
df_lexicol_eval.to_csv('data/analysis/lexicol_eval.csv')

# levenstein_ops_stif and leexicol, save them as json
import json

with open('data/analysis/stif-indo_eval.json', 'w+') as file:
    json.dump(levenstein_ops_stif, file)

with open('data/analysis/lexicol_eval.json', 'w+') as file:
    json.dump(levenstein_ops_lexicol, file)